# Analisis de Egresos Hospitalarios

In [6]:
import polars as pl
import glob

DICT_VARIABLES = {
    'ESTABLECIMIENTO_SALUD': pl.Int32,
    'GLOSA_ESTABLECIMIENTO_SALUD': pl.Categorical,
    'PERTENENCIA_ESTABLECIMIENTO_SALUD': pl.Categorical,
    'SEREMI': pl.Int8,
    'SERVICIO_DE_SALUD': pl.Int8,
    'SEXO': pl.Int8,
    'EDAD_CANT': pl.Int8,
    'TIPO_EDAD': pl.Int8,
    'EDAD_A_OS': pl.Int8,
    'PUEBLO_ORIGINARIO': pl.Int8,
    'PAIS_ORIGEN': pl.Int16,
    'GLOSA_COMUNA_RESIDENCIA': pl.Categorical,
    'REGION_RESIDENCIA': pl.Categorical,
    'GLOSA_REGION_RESIDENCIA': pl.Categorical,
    'PREVISION': pl.Int8,
    'BENEFICIARIO': pl.Categorical,
    'MODALIDAD': pl.Int8,
    'PROCEDENCIA': pl.Int8,
    'ANO_EGRESO': pl.Int16,
    'FECHA_EGRESO': pl.Date,
    'AREA_FUNCIONAL_EGRESO': pl.Int16,
    'DIAS_ESTADA': pl.Int16,
    'CONDICION_EGRESO': pl.Int8,
    'INTERV_Q': pl.Int8,
    'CODIGO_INTERV_Q_PPAL': pl.Int32,
}

HOSPITAL_A_ANALIZAR = 112103 # Este es el Torax

In [2]:
def obtener_diagnosticos_unicos_de_hospital(hospital_a_analizar):
  diags_hospital = (
      pl.scan_csv('input/utf-8/*.csv', separator=';')
        .filter(pl.col('ESTABLECIMIENTO_SALUD') == hospital_a_analizar)
        .select(pl.col('DIAG1')).unique()
        .collect(streaming=True)
  ).to_series()

  return diags_hospital

diags_torax = obtener_diagnosticos_unicos_de_hospital(HOSPITAL_A_ANALIZAR)

In [3]:
agrupacion_anio = ['ANO_EGRESO', 'ESTABLECIMIENTO_SALUD', 'GLOSA_ESTABLECIMIENTO_SALUD', 'DIAG1']

metricas_agregadas = (
    pl.scan_csv('input/utf-8/*.csv', separator=';')
      .filter(pl.col('DIAG1').is_in(diags_torax))
      .groupby(agrupacion_anio).agg(
          [pl.col('DIAG1').count().alias('n_egresos'),
           pl.col('DIAS_ESTADA').mean().alias('dias_estada_promedio'),
           pl.col('INTERV_Q').sum().alias('n_int_q'),
           pl.col('CONDICION_EGRESO').sum().alias('n_muertos')]
      )
)

In [4]:
df = pl.read_csv('input/utf-8/Egresos_Hospitalarios_2014.csv', dtypes=DICT_VARIABLES, separator=';')

In [5]:
df['GLOSA_ESTABLECIMIENTO_SALUD'].value_counts()

GLOSA_ESTABLECIMIENTO_SALUD,counts
cat,u32
"""Hospital Clíni…",8251
"""Hospital Milit…",3085
"""Hospital de Pa…",160
"""Hospital San J…",1185
"""Hospital Carlo…",25973
"""Clínica Recupe…",86
"""Hospital de Te…",997
"""Hospital Dr. M…",2190
"""Hospital Clíni…",1382


In [22]:
df = df.with_columns([
    pl.col('INTERV_Q').map_dict({2: 0}, default=pl.col('INTERV_Q')).alias('INTERV_Q'),
    pl.col('CONDICION_EGRESO').map_dict({1: 0, 2: 1}, 
                                        default=pl.col('CONDICION_EGRESO'))
                                        .alias('CONDICION_EGRESO')
])

In [6]:
import pandas as pd
import numpy as np

import geopandas as gpd
import geopy

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [25]:
locator = Nominatim(user_agent='javier', timeout=20)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001)

def location_info(x):
    data = locator.geocode(x).raw
    data_converted = pd.json_normalize(data).squeeze()
    return data_converted

In [3]:
df = pd.read_excel('output/distribucion_sociodemografica/distribucion_sociodemografica.xlsx')

In [43]:
df['region_pais'] = 'Region ' + df['GLOSA_REGION_RESIDENCIA'] + ', Chile'
df['comuna_region_pais'] = df['GLOSA_COMUNA_RESIDENCIA'] + ', ' + df['region_pais']

In [47]:
regiones_unicas = pd.Series(df['region_pais'].unique()).apply(location_info)

AttributeError: 'NoneType' object has no attribute 'raw'

In [52]:
sorted(df['GLOSA_REGION_RESIDENCIA'].unique())

['De Aisén del Gral. C. Ibáñez del Campo',
 'De Aisï¿½n del Gral. C. Ibï¿½ï¿½ez del Campo',
 'De Antofagasta',
 'De Arica y Parinacota',
 'De Atacama',
 'De Coquimbo',
 'De La Araucanía',
 'De La Araucanï¿½a',
 'De Los Lagos',
 'De Los Ríos',
 'De Los Rï¿½os',
 'De Magallanes y de La Antártica Chilena',
 'De Magallanes y de La Antï¿½rtica Chilena',
 'De Tarapacá',
 'De Tarapacï¿½',
 'De Valparaíso',
 'De Valparaï¿½so',
 'De Ñuble',
 'De ï¿½uble',
 'Del Bíobío',
 'Del Bï¿½obï¿½o',
 "Del Libertador B. O'Higgins",
 'Del Maule',
 'Ignorada',
 'Metropolitana de Santiago']

In [ ]:
with open('input/Egresos_Hospitalarios_2014.csv', encoding='latin-1') as f:
    df = pl.read_csv(f.read())

In [58]:
df = pl.read_csv('input/Egresos_Hospitalarios_2014.csv', separator=';', dtypes=DICT_VARIABLES,
                 use_pyarrow=False)

In [107]:
with open('input/Egresos_Hospitalarios_2014.csv', encoding='utf-8') as f:
    df = pl.read_csv(f.read().encode('utf-8'), separator=';', dtypes=DICT_VARIABLES)

In [108]:
df

ID_PACIENTE,ESTABLECIMIENTO_SALUD,GLOSA_ESTABLECIMIENTO_SALUD,PERTENENCIA_ESTABLECIMIENTO_SALUD,SEREMI,SERVICIO_DE_SALUD,SEXO,FECHA_NACIMIENTO,EDAD_CANT,TIPO_EDAD,EDAD_A_OS,PUEBLO_ORIGINARIO,PAIS_ORIGEN,GLOSA_PAIS_ORIGEN,COMUNA_RESIDENCIA,GLOSA_COMUNA_RESIDENCIA,REGION_RESIDENCIA,GLOSA_REGION_RESIDENCIA,PREVISION,BENEFICIARIO,MODALIDAD,PROCEDENCIA,ANO_EGRESO,FECHA_EGRESO,AREA_FUNCIONAL_EGRESO,DIAS_ESTADA,CONDICION_EGRESO,DIAG1,GLOSA_DIAG1,DIAG2,GLOSA_DIAG2,INTERV_Q,CODIGO_INTERV_Q_PPAL,GLOSA_INTERV_Q_PPAL,PROCED,CODIGO_PROCED_PPAL,GLOSA_PROCED_PPAL
str,i32,cat,cat,i8,i8,i8,str,i8,i8,i8,i8,i16,str,i64,cat,cat,cat,i8,cat,i8,i8,i16,date,i16,i16,i8,str,str,str,str,i8,i32,str,str,str,str
"""NA""",112101,"""Hospital Dr. L…","""Pertenecientes…",null,12,2,"""2014-12-08""",2,3,0,96,152,"""Chile""",13122,"""Pe�alol�n""","""13""","""Metropolitana …",99,null,null,1,2014,2014-12-10,324,2,1,"""P221""","""TAQUIPNEA TRAN…",null,null,2,null,null,null,null,null
"""NA""",112101,"""Hospital Dr. L…","""Pertenecientes…",null,12,1,"""2014-12-08""",5,3,0,96,152,"""Chile""",15101,"""Arica""","""15""","""De Arica y Par…",99,null,null,1,2014,2014-12-13,152,5,1,"""P221""","""TAQUIPNEA TRAN…",null,null,2,null,null,null,null,null
"""NA""",112101,"""Hospital Dr. L…","""Pertenecientes…",null,12,1,"""2014-12-05""",5,3,0,96,152,"""Chile""",13122,"""Pe�alol�n""","""13""","""Metropolitana …",99,null,null,5,2014,2014-12-10,152,2,1,"""P598""","""ICTERICIA NEON…",null,null,2,null,null,null,null,null
"""NA""",112101,"""Hospital Dr. L…","""Pertenecientes…",null,12,1,"""2014-12-09""",13,3,0,96,152,"""Chile""",13115,"""Lo Barnechea""","""13""","""Metropolitana …",99,null,null,1,2014,2014-12-22,324,13,1,"""P590""","""ICTERICIA NEON…",null,null,2,null,null,null,null,null
"""NA""",112101,"""Hospital Dr. L…","""Pertenecientes…",null,12,2,"""2014-12-07""",4,3,0,96,152,"""Chile""",13122,"""Pe�alol�n""","""13""","""Metropolitana …",99,null,null,5,2014,2014-12-11,152,2,1,"""P598""","""ICTERICIA NEON…",null,null,2,null,null,null,null,null
"""NA""",112101,"""Hospital Dr. L…","""Pertenecientes…",null,12,2,"""2014-12-09""",2,3,0,96,152,"""Chile""",13122,"""Pe�alol�n""","""13""","""Metropolitana …",99,null,null,5,2014,2014-12-11,152,2,1,"""Q185""","""MICROSTOMIA""",null,null,2,null,null,null,null,null
"""NA""",107224,"""Hospital Cl�ni…","""No Pertenecien…",5,null,2,"""2014-01-22""",2,3,0,96,152,"""Chile""",5804,"""Villa Alemana""","""05""","""De Valpara�so""",2,null,null,5,2014,2014-01-25,330,1,1,"""P599""","""ICTERICIA NEON…",null,null,2,null,null,null,null,null
"""NA""",107224,"""Hospital Cl�ni…","""No Pertenecien…",5,null,1,"""2014-01-14""",2,3,0,96,152,"""Chile""",5502,"""Calera""","""05""","""De Valpara�so""",1,"""C""",2,5,2014,2014-01-26,311,10,1,"""P239""","""NEUMONIA CONGE…",null,null,2,null,null,null,null,null
"""NA""",101100,"""Hospital Dr. J…","""Pertenecientes…",null,1,2,"""1928-01-21""",86,1,86,96,152,"""Chile""",15101,"""Arica""","""15""","""De Arica y Par…",1,"""B""",1,1,2014,2014-02-19,110,4,1,"""I509""","""INSUFICIENCIA …",null,null,2,null,null,null,null,null


In [77]:
import chardet

In [104]:
text = open('input/Egresos_Hospitalarios_2001.csv', 'rb').read(1000)
print(chardet.detect(text))

{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}


In [ ]:
with open('input/Egresos_Hospitalarios_2014.csv', encoding=)